In [ ]:
#CODE FOR INITIALIZING REWARD TABLE
import pickle
from pprint import pprint as pp
import datetime
import glob
import random
import os
from textblob import TextBlob
import pandas as pd
import pickle
import os

dir='C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'
strategies_dict=pickle.load( open("C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/strategies.pkl", "rb" ) )

N_TURN = 3
list1=[]
bot_strategies=[]
learning_rate=0.01
discount_factor=0.9
epsilon=0.9

user_utterances=[]
strategies_list=[]

evaluation_dict={}
user_lines=[]
#state action table
R_table=pd.DataFrame.from_items([('qs+b', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("qs+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("qs'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                            ("q'ls-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("q'ls-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("q'l's+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                          ("q'l's'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's'e", [0.0, 0.0, 0.0,0.0,0.0,0.0])],
                                            orient='index', 
                                          columns=['question', 'elaborate', 'opinion','affirmative','negative','neutral'])

#temporary dictionary which stores the count of the recorded utterance till date,this is used for calculating the average to be stored in state table
temp_dict={"qs+b":0,"qs+e":0,"qs-b":0,"qs-e":0,"qs'b":0,"qs'e":0,"q'ls+b":0,"q'ls+e":0,"q'ls-b":0,"q'ls-e":0,"q'ls'b":0,"q'ls'e":0,
           "q'l's+b":0,"q'l's+e":0,"q'l's-b":0,"q'l's-e":0,"q'l's'b":0,"q'l's'e":0}

Q_table=pd.DataFrame.from_items([('qs+b', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("qs+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("qs'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                            ("q'ls-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("q'ls-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("q'l's+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                          ("q'l's'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's'e", [0.0, 0.0, 0.0,0.0,0.0,0.0])],
                                            orient='index', 
                                          columns=['question', 'elaborate', 'opinion','affirmative','negative','neutral'])

In [ ]:
#utility functions
def check_strategy(response):
    for key, value in strategies_dict.items():
        if response in value:
            #print(key)
            break
            
    return key
def check_strategies(response):
    temp_list=response.split(":")
    temp_list2=temp_list[0].split('_')
    return temp_list2[1]
    

def get_utterance(line):
    temp_list=line.split(":")
    value=temp_list[3]
    return value

def populate_evaluation(line):
    list1=line.split(',')
    for values in list1:
        if values is not '':
            list2=values.split('=')
            evaluation_dict[list2[0]]=list2[1]

#utility functions
def next_state(utterance_no):
    if not utterance_no==len(user_utterances)-1:
        return user_utterances[utterance_no+1]
    else:
        return 0
    
def max_q_value(utterance):
    max=0
    for column in Q_table:
        if Q_table.at[utterance,column]>max:
            max=Q_table.at[utterance,column]
    return max
max_strat=''
def max_strategy(utterance):
    value=''
    max=0
    for column in Q_table:
        if Q_table.at[utterance,column]>max:
            max=Q_table.at[utterance,column]
            value=column
    return value
def training(indexes):
    
    for line in user_utterances:
        next_utterance=next_state(indexes)
        prev_value=Q_table.at[user_utterances[indexes],strategies_list[indexes]]
        if next_utterance is not 0:
            Q_table.at[user_utterances[indexes],strategies_list[indexes]]=((temp_dict[user_utterances[indexes]]-1)*Q_table.at[user_utterances[indexes],strategies_list[indexes]]+R_table.at[user_utterances[indexes],strategies_list[indexes]] + discount_factor*max_q_value(next_utterance))/temp_dict[user_utterances[indexes]] 
        else:
            Q_table.at[user_utterances[indexes],strategies_list[indexes]]=R_table.at[user_utterances[indexes],strategies_list[indexes]]

#def training_learning():
    

#def learn():
    

In [ ]:
list1=[]
user_utterances=[]
bot_strategies=[]
#function which updates the state value table for each conversation where list1 is the utterance of the user and list2 is the strategy
#applied by the bot
def record_check(list1,list2,evaluated_score):
    i=0
    for key in list1:
        #print state_action_dict.at[key,list2[i]]
        previous_value=state_action_dict.at[key,list2[i]]
        temp_dict[key]=temp_dict[key]+1
        #print(float(evaluated_score))
        #print temp_dict[key],((previous_value*(temp_dict[key]-1))+evaluated_score)/temp_dict[key]
        value=((previous_value*(temp_dict[key]-1))+float(evaluated_score))/temp_dict[key]
        #print(value)
        state_action_dict.at[key,list2[i]]=value
        i=i+1

def question_or_not(words):
    if "?" not in words:
        q="q'"
    else:
        q="q"
    #print q
    return q

def count_no_words(words):
    no=len(words.split())
    if no>=10:
        #print "long"
        l="l"
    else:
        #print "short"
        l="l'"
    return l

def sentiment(words):
    blob = TextBlob(words)
    senti=blob.sentiment
    if senti.polarity<0.2 and senti.polarity>-0.2:
        s="s'"
    else:
        if senti.polarity>0:
            s="s+"
        else:
            s="s-"
    return s
    


def beginning_end(line,list):
    length=len(list)
    if list.index(line)==0:
        #print "beginning"
        var="b"
    else:
        #print "ending"
        var="e"
    return var

def rows_utterance(list1):
    
    for values in list1:
        if question_or_not(values)=="q":
            if sentiment(values)=="s+":
                if beginning_end(values,list1)=="b":
                    result="qs+b"
                else:
                    result="qs-e"
            elif sentiment(values)=="s-":
                if beginning_end(values,list1)=="b":
                    result="qs-b"
                else:
                    result="qs-e"
            else:
                if beginning_end(values,list1)=="b":
                    result="qs'b"
                else:
                    result="qs'e"
        else:
            if count_no_words(values)=="l":
            
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'ls+b"
                    else:
                        result="q'ls+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'ls-b"
                    else:
                        result="q'ls-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'ls'b"
                    else:
                        result="q'ls'e"
            else:
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'l's+b"
                    else:
                        result="q'l's+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'l's-b"
                    else:
                        result="q'l's-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'l's'b"
                    else:
                        result="q'l's'e"
        user_utterances.append(result)
    return user_utterances

def single_utterance(values):
    if question_or_not(values)=="q":
        if sentiment(values)=="s+":
            result="qs+e"
        elif sentiment(values)=="s-":
            result="qs-e"
        else:
            result="qs'e"
    else:
        if count_no_words(values)=="l":
            
            if sentiment(values)=="s+":
                
                result="q'ls+e"
            elif sentiment(values)=="s-":
                result="q'ls-e"
            else:
                result="q'ls'e"
        else:
            if sentiment(values)=="s+":
                
                result="q'l's+e"
            elif sentiment(values)=="s-":
                result="q'l's-e"
            else:
                result="q'l's'e"
        
    return result



In [ ]:
#CODE FOR EXTRACTING VALUES FROM REWARD TABLE AND LEARNING FROM IT

user_utterances=[]

for filename in os.listdir(dir):
    
    file = open('C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'+filename,'r')
    strategies_list=[]
    user_lines=[]
    user_utterances=[]
    for lines in file:
        #print(lines)
        if not lines.startswith(("overall","engaing","return","interupt","start")):
            if lines.startswith('Bot'):
                strategies_list.insert(len(strategies_list),check_strategies(lines))
            else:
                user_lines.append(get_utterance(lines))
                
        else:
            strategies_list=strategies_list[1:]
            if lines.endswith(','):
                populate_evaluation(lines)
                user_utterances=rows_utterance(user_lines)
                user_utterances=user_utterances[:len(user_utterances)-1]
                
                temp_dict[user_utterances[0]]=temp_dict[user_utterances[0]]+1
                R_table.at[user_utterances[0],strategies_list[0]]=((temp_dict[user_utterances[0]]-1)*R_table.at[user_utterances[0],strategies_list[0]]+0.8*float(evaluation_dict['start'])+0.2*float(evaluation_dict['overall']))/temp_dict[user_utterances[0]]
                count=1
                for line in user_utterances[1:len(user_utterances)]:
                    #print(count)
                    temp_dict[user_utterances[count]]=temp_dict[user_utterances[count]]+1
                    R_table.at[user_utterances[count],strategies_list[count]]=((temp_dict[user_utterances[count]]-1)*R_table.at[user_utterances[count],strategies_list[count]]+0.4*float(evaluation_dict['engaing'])+0.4*float(evaluation_dict['interupt'])+0.2*float(evaluation_dict['overall']))/temp_dict[user_utterances[count]]               
                    count=count+1
            else:
                pass
        
    file.close()

In [ ]:
#for training from the model
for filename in os.listdir(dir):
    
    file = open('C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'+filename,'r')
    strategies_list=[]
    user_lines=[]
    user_utterances=[]
    for lines in file:
        #print(lines)
        if not lines.startswith(("overall","engaing","return","interupt","start")):
            if lines.startswith('Bot'):
                strategies_list.insert(len(strategies_list),check_strategies(lines))
            else:
                user_lines.append(get_utterance(lines))
    user_utterances=rows_utterance(user_lines)
    strategies_list=strategies_list[1:]
    user_utterances=user_utterances[:len(user_utterances)-1]
    for values in user_utterances:
        indexes=user_utterances.index(values)
        training(indexes)

In [ ]:
import pickle
from pprint import pprint as pp
import datetime
import glob
import random
import os

N_TURN = random.choice(range(5,11)) 
DATA_PATH = "C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/new_convo/"
strats = pickle.load(open("C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/strategies.pkl", "rb"))

def main():
    record_convo()
    while input("Do you want to record another conversation? (Y/N)").lower() == "y":
        record_convo()
        

def record_convo():
    # load strategies from pickle
    #strats = pickle.load(open("strategies.pkl", "rb"))
    list1=[]
    try:
        # create file
        name = input("Before we start: what's your name?\n")
        date = str(datetime.datetime.today().date())
        convo_num = get_convo_num(name)
        filename = DATA_PATH + "{0}_{1}_{2}.txt".format(name.lower(), date, convo_num) 
        f = open(filename, "w")
        
        # record converstaion
        print("----------------------------Here it gones--------------------------------")
        #strategy=Q_table.iloc[0].argmax()
        strategy='question'
        utterance=random.choice(list(strats[strategy]))
        log("Bot", utterance, strategy, f)
        strats[strategy].remove(utterance)
        for turn in range(N_TURN-1):
            answer=str(input(name + ": "))
            log(name, answer, "None",f)
            
            utter=single_utterance(answer)
            strategy=''
            strategy=max_strategy(utter)
            utter=''
            utterance=random.choice(list(strats[strategy]))
            
            log("Bot", utterance, strategy, f)
            strats[strategy].remove(utterance)
        answer=str(input(name + ": "))
        log(name, answer, "None",f)    
        print("-----------------------------The end-------------------------------------")

        # evaluation and wrap up
        scores = get_evaluation()
        for score_type, score_value in scores.items():
            f.write("{0}={1},".format(score_type, score_value))
        f.close()
        print("Done!")
    
    except:
        os.remove(filename)
        raise    


def get_evaluation():
    scores = {}
    questions = {"overall": "How is the conveseration overall? (0 is aweful, 5 is amazing)\n",
                 "start": "How is the begining of the converstaion? (0: weird and out of context, 5: natural)\n",
                 "interupt": "How is the continuity of the conversation? (0: you get interupted too much, 5: very fluid and coherent)\n",
                 "engaing": "How engaging or interesting is the conversation? (0 is boring, 5 is very interesting)\n",
                 "return": "Would you like to talk to this bot again? (0 is not at all, 5 is definitely)\n"}
    for score_type in questions.keys():
        score = input(questions[score_type])
        while len(score) > 1 or ord(score) < ord("0") or ord(score) > ord("5"):
            score = input("Type a integer between [0,5], otherwise I will keep asking")
        scores[score_type] = score
    return scores


def get_convo_num(name):
    try:
        files = glob.glob(DATA_PATH + name.lower() + "_*")
        existing_nums = list(map(int, [i[:-4].split("_")[-1] for i in files]))
        return max(existing_nums) + 1
    except ValueError:
        return 1
    

def log(role, utter, strategy, file):
    if role == "Bot":
        print("Bot says: ", utter)
    time = datetime.datetime.now().strftime("%H:%M:%S")
    file.write("{0}_{1}_{2}:{3}\n".format(role, strategy, time, utter))


